In [1]:
from keras import Input, layers

input_tensor = Input(shape=(32,))
dense = layers.Dense(32, activation='relu')

output_tensor = dense(input_tensor)


In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train)


Epoch 1/10
8/8 [==============================] - 0s 1ms/step - loss: 11.6292
Epoch 2/10
8/8 [==============================] - 0s 1ms/step - loss: 11.9710
Epoch 3/10
8/8 [==============================] - 0s 2ms/step - loss: 12.8654
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 13.9987
Epoch 5/10
8/8 [==============================] - 0s 3ms/step - loss: 15.5406
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 17.3803
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 19.4441
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 21.5213
Epoch 9/10
8/8 [==============================] - 0s 2ms/step - loss: 23.9958
Epoch 10/10
32/32 [==============================] - 0s 768us/step - loss: 27.3114
